In [13]:
import numpy as np

get_cutoff = lambda R, C: 1 / (2 * np.pi * R * C)

For f_min GND at VCO_1,IN

3.1.1.1 a

In [14]:
V_p_p = 5
R_2 = 100e3
C_1 = 220e-12

R_1 =   [22e3,   10e3,   47e3,   68e3,   100e3,  220e3,  470e3,  1e6]
f_min = [52.1e3, 52.1e3, 52.1e3, 52.1e3, 52.1e3, 52.1e3, 52.1e3, 52.1e3]

3.1.1.1 b

In [15]:
V_p_p = 5
R_1 = 22e3
C_1 = 220e-12

R_2 =   [10e3,  22e3,  47e3,   68e3,   100e3, 220e3,  470e3,  1e6]
f_min = [296e3, 177e3, 98.7e3, 72.5e3, 52e3,  22.6e3, 12.5e3, 6.1e3]

3.1.1.1 c

In [16]:
V_p_p = 5
R_1 = 22e3
R_2 = 47e3

C_1 =   [56e-12, 220e-12, 470e-12, 1e-9, 4.7e-9, 22e-9, 100e-9, 220e-9]
f_min = [295e3,  98e3,    49e3,    24e3, 5.3e3,  1.2e3, 263   , 112]

3.1.1.2

For f_max 5V on VCO_1_IN

In [17]:
R_2 = 47e3
C_1 = 220e-12

R_1 =   [22e3,  10e3,  47e3,  68e3,  100e3, 220e3, 470e3, 1e6]
f_max = [237e3, 345e3, 172e3, 151e3, 135e3, 116e3, 107e3, 103e3]

3.1.3

3.1.2.1

In [18]:
R_1 = 22e3
R_2 = 100e3
C_1 = 220e-12

V_in =  [0,     0.5,    1,      1.5,  2,    2.5,   3,     3.5,   4.0,   4.5,   5]
f_vco = [52.e3, 52.1e3, 52.1e3, 57e3, 82e3, 112e3, 141e3, 170e3, 199e3, 208e3, 209e3]

3.1.3

In [20]:
R_f  = 56e3
C_f = 1e-9

f_c = get_cutoff(R_f, C_f)
print(f"{f_c} Hz")

f_H_min = 55e3
f_H_max = 193e3

f_f_min = 87e3
f_f_max = 138e3

2842.052555212417 Hz


3.1.3.2

In [22]:
R_f_max = 100e3
C_f_min = 5.6e-9

f_c_min = get_cutoff(R_f_max, C_f_min)
print(f_c_min)

f_H_max = 200e3
f_H_min = 55e3
f_f_max = 123e3
f_f_min = 105e3

284.2052555212417


3.1.3.3

In [23]:
R_f_min = 10e3
C_f_min = 560e-12

f_c_max = get_cutoff(R_f_min, C_f_min)
print(f_c_max)

f_H_max = 200e3
f_f_max = 200e3
f_H_min = 55e3
f_f_min = 56e3

28420.525552124167


3.1.3.5

In [24]:
R_f = 10e3
C_f = 1.8e-9

f_H_max = 200e3
f_f_max = 158e3
f_H_min = 55e3
f_f_min = 67e3

3.2.2 a

In [ ]:
RBW = 1e3
VBW = 30
span = 50e3

freq_offset = [1e3, 2e3, 5e3, 10e3, 25e3]



3.2.2 b